In [1]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
from time import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import scipy
from sklearn import svm
from sklearn.metrics import confusion_matrix
from multiprocessing import Pool

import random
from itertools import product

In [2]:
X = np.load('/oasis/projects/nsf/csd395/ruogu/svm2/X.npy');
y = np.load('/oasis/projects/nsf/csd395/ruogu/svm2/y.npy');
print "%s" %(X.shape,);
print "%s" %(y.shape,);
raw_data = zip(X, y);
print type(raw_data);

(188118, 4200)
(188118,)
<type 'list'>


In [3]:
data = [];
n_classSample = 1000; # Number of samples from each class
for i in range(0, 10):
    data_for_class = [x for x in raw_data if x[1] == i];
    data = data + random.sample(data_for_class, n_classSample);
# data is a list of (ndarray, int)
random.shuffle(data);
quota = 0.75;
n_training = int(len(data) * quota);
n_testing = len(data) - n_training;

In [4]:
training_data = data[:n_training];
testing_data = data[n_training:];

In [5]:
training_X = np.asarray([x[0] for x in training_data]);
training_y = np.asarray([x[1] for x in training_data]);
testing_X = np.asarray([x[0] for x in testing_data]);
testing_y = np.asarray([x[1] for x in testing_data]);

In [6]:
print "Shape of training data: %s" %(training_X.shape,);
print "Shape of training label: %s" %(training_y.shape,);
print "Shape of testing data: %s" %(testing_X.shape,);
print "Shape of testing label: %s" %(testing_y.shape,);

Shape of training data: (7500, 4200)
Shape of training label: (7500,)
Shape of testing data: (2500, 4200)
Shape of testing label: (2500,)


In [18]:
# serial version
n_train = training_X.shape[0]
a = np.empty((n_train, n_train))

import time
t = time.time()
for i in range(n_train)[:1000]:
    a[i] = .5 * np.sum(training_X + training_X[i] - np.abs(training_X - training_X[i]), axis=1)
print time.time() - t

44.0860569477


In [21]:
# parallel version
from joblib import Parallel, delayed

def f(i):
    return .5 * np.sum(training_X + training_X[i] - np.abs(training_X - training_X[i]), axis=1)

import time
t = time.time()

n_train = training_X.shape[0]
res = Parallel(n_jobs=16)(delayed(f)(i) for i in range(n_train))

print time.time() - t